In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle
import numpy as np
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi
from exact_sync.v1.api.teams_api import TeamsApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [4]:
configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = '****'
configuration.host = "https://exact.cs.fau.de/"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [5]:
image_set = image_sets_api.retrieve_image_set(id=246)
image_set

{'creator': 1,
 'description': None,
 'id': 246,
 'images': [3573,
            3571,
            3568,
            3566,
            3564,
            3562,
            3560,
            3558,
            3556,
            3554,
            3551,
            3549,
            3547,
            3545,
            3543,
            3539,
            3536,
            3530],
 'location': None,
 'main_annotation_type': None,
 'name': 'Berliner Blau ExpertAlgo',
 'path': 'imagetagger_4_246',
 'product_set': [3],
 'set_tags': [],
 'team': 4}

In [ ]:
annotations = []

for image_id in tqdm(image_set.images):
    for anno in annotations_api.list_annotations(image=image_id, pagination=False, fields="id,vector,annotation_type,deleted,last_editor").results:
        if  anno.vector is not None and "x2" in anno.vector and "x1" in anno.vector and "y1" in anno.vector and "y2" in anno.vector:
            annotations.append([anno.id, image_id, anno.last_editor, anno.deleted, anno.vector["x2"] - anno.vector["x1"], anno.vector["y2"] - anno.vector["y1"]])

In [8]:
df =  pd.DataFrame(annotations, columns=['id', 'image', 'editor', 'deleted', 'width', 'height'])
df.head()

,id,image,editor,deleted,width,height
0,1524574,3573,1,False,71,69
1,1524575,3573,1,False,89,89
2,1524576,3573,1,False,70,70
3,1524577,3573,1,False,101,101
4,1524578,3573,1,False,89,89


In [9]:
screened = [3560, 3558, 2556, 2554, 2551, 3549, 3545, 3543, 3539, 3536, 3530, 3573, 3512]

In [13]:
deleted_cells = df[df['deleted'] == True]
undeleted_cells = df[df['deleted'] == False]

In [14]:
deleted_cells.shape

(5599, 6)

In [16]:
undeleted_cells.shape

(78936, 6)

In [12]:
image_cells = df[df['image'] == 3556]
fig = px.scatter(image_cells, x="width", y="height", color="deleted")
fig.show()